In [1]:
import sys
import os
import datetime
sys.path.insert(0, os.environ['VIRTUAL_ENV']+'/ghtesting')

import matplotlib.pyplot as plt
# plt.rcParams["figure.figsize"]=10,10
import pandas as pd
plt.style.use(['seaborn'])
print(plt.style.available)

['Solarize_Light2', '_classic_test_patch', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn', 'seaborn-bright', 'seaborn-colorblind', 'seaborn-dark', 'seaborn-dark-palette', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'tableau-colorblind10']


In [2]:
def days_since_date(datestr):
    date = pd.to_datetime(datestr, utc=True)
    today = pd.to_datetime(datetime.datetime.utcnow().isoformat(), utc=True)
    return (today - date).days

In [3]:
from ghdatabase import GHDatabase
from ghrepo import GHRepo

db = GHDatabase('github', 'webframework_repos')

In [20]:
df = pd.DataFrame()
webtopics = ['angular', 'react', 'vue']
for t in webtopics:
    new_row = dict()
    new_row['framework'] = t
    for repojson in db.get_repos():
        repo = GHRepo(repojson)
        if repo.stars is None:
            continue
        if repo.isfork:
            continue # there are none
        if repo.primarylanguage == 'n/a':
            continue
        if repo.funding_links is None:
            continue
        if t not in repo.topics:
            continue
    #     print(repo.url)
    #     print(repo.badges[0])

        push_age = days_since_date(repo.pushed_at)
        create_age = days_since_date(repo.created_at)
        if push_age > 730:
            continue # filter out repos that are too old
        if len(repo.badges[0]) > 1:
            continue
        for b in repo.badges[0]:
            new_row[b] = new_row.get(b, 0) + 1

    df = df.append(new_row, ignore_index=True)
df = df.fillna(0)
df

,framework,github,travisci,circleci,azure_pipelines,appveyorci,semaphoreci,gitlab,saucelabs,bitrise,buildkite,codeship,shippable,teamcity,wercker
0,angular,115.0,216.0,66.0,12.0,3.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,react,473.0,758.0,236.0,21.0,4.0,3.0,4.0,2.0,1.0,3.0,2.0,1.0,2.0,1.0
2,vue,185.0,274.0,100.0,12.0,3.0,2.0,3.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0


In [50]:
from scipy.stats import chi2_contingency
# topic_5ci = df[['framework', 'travisci', 'github', 'circleci', 'azure_pipelines', 'appveyorci']].copy()
topic_5ci = df[['framework', 'travisci', 'github', 'circleci', 'azure_pipelines']].copy()

# topic_5ci = df[['framework', 'travisci', 'github', 'circleci']].copy()
topic_5ci.set_index('framework', inplace=True)

topic_5ci.astype(int)



,travisci,github,circleci,azure_pipelines
framework,,,,
angular,216,115,66,12
react,758,473,236,21
vue,274,185,100,12


### Test of Independence
- H0: framework and ci service are independent

- H1: framework and ci service are dependent

since p > .05, fail to reject the null

In [51]:
chi2_contingency(topic_5ci)


(7.950630189620748,
 0.2417426307284076,
 6,
 array([[206.82009724, 128.10251216,  66.61993517,   7.45745543],
        [752.44084279, 466.05510535, 242.37277147,  27.13128039],
        [288.73905997, 178.8423825 ,  93.00729335,  10.41126418]]))

In [52]:
# topic_5ci['Total'] = topic_5ci.sum(axis=1)
print(topic_5ci.to_latex())

\begin{tabular}{lrrrr}
\toprule
{} &  travisci &  github &  circleci &  azure\_pipelines \\
framework &           &         &           &                  \\
\midrule
angular   &     216.0 &   115.0 &      66.0 &             12.0 \\
react     &     758.0 &   473.0 &     236.0 &             21.0 \\
vue       &     274.0 &   185.0 &     100.0 &             12.0 \\
\bottomrule
\end{tabular}



In [55]:
topic_5ci.sum()

travisci           1248.0
github              773.0
circleci            402.0
azure_pipelines      45.0
dtype: float64